<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Utility" data-toc-modified-id="Utility-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Utility</a></span></li><li><span><a href="#Feature-Scaling" data-toc-modified-id="Feature-Scaling-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature Scaling</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#one-vs-rest" data-toc-modified-id="one-vs-rest-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>one-vs-rest</a></span></li><li><span><a href="#radius_neighbors-Approach" data-toc-modified-id="radius_neighbors-Approach-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>radius_neighbors Approach</a></span><ul class="toc-item"><li><span><a href="#Multiple-products" data-toc-modified-id="Multiple-products-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Multiple products</a></span></li><li><span><a href="#All-products" data-toc-modified-id="All-products-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>All products</a></span></li></ul></li><li><span><a href="#kneighbors-approach" data-toc-modified-id="kneighbors-approach-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>kneighbors approach</a></span><ul class="toc-item"><li><span><a href="#Single-product" data-toc-modified-id="Single-product-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Single product</a></span></li><li><span><a href="#Multiple-products" data-toc-modified-id="Multiple-products-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Multiple products</a></span></li><li><span><a href="#All-products" data-toc-modified-id="All-products-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>All products</a></span></li></ul></li></ul></li></ul></div>

# Setup

In [ ]:
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip -V

/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory


In [ ]:
# setup to see the execution time in each cell

!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install ipython-autotime
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install directory_structure
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install handcalcs
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install pytz
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install skimage
# !/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install opencv-contrib-python==4.4.0.44
# %load_ext autotime

/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory


In [9]:
import os
import random
import glob
import cv2
import shutil
import pickle
import PIL
import pandas as pd
from PIL import Image
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# import cupy as cp

# from google.colab.patches import cv2_imshow
# from directory_structure import Tree

In [10]:
print (cv2.__version__)

4.1.2


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# replace the dir if needed

ROOT_DIR = "/content/drive/MyDrive/OneCareer/Project - G5/Dataset/cv"  # Henry's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset/"    #Lejia's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset/" #Nan's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset"
# ROOT_DIR ="/content/drive/MyDrive/Project - G5/Dataset"


# path for image augmentation
RESIZE_DIR = ROOT_DIR + "/augmentation/resize"


# path for the different handcraft features
RAW_PIXEL_DIR = ROOT_DIR + "/handcraft_features/raw_pixel"
SIFT_DIR = ROOT_DIR + "/handcraft_features/sift"
HOG_DIR = ROOT_DIR + "/handcraft_features/hog"
LBP_DIR = ROOT_DIR + "/handcraft_features/lbp"


# menu datasets
MENU_FOOD = ROOT_DIR + "/MeetFresh_menu_food_2.csv"
MENU_DRINK = ROOT_DIR + "/MeetFresh_menu_drink_2.csv"

## Utility

Retrieve local descriptor

In [13]:
def retrieve_stored_features(path: str, product_name: str):
    # retrieve a stored local descriptor based on given name
    try:
        file_path = path + "/" + product_name + ".txt"
        file = open(file_path, 'rb')
        descriptor = pickle.load(file)
        file.close()

        return descriptor

    except FileNotFoundError:
        print("Wrong file name or path for {}".format(product_name))
        return None
    except e:
        print("Error due to: {}".format(e))
        return None

In [14]:
files = os.listdir(HOG_DIR)
x = list()

for file in files:
    x.append(retrieve_stored_features(HOG_DIR, file.split(".")[0]))

print(">>> All jobs done!")

>>> All jobs done!


## Feature Scaling

Feature Scaling before fitting the data into the model.

In [15]:
sc = StandardScaler()
x = sc.fit_transform(x)

# Model

In [16]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')

In [17]:
index = 0

In [18]:
model_knn.fit(x)

NearestNeighbors(algorithm='brute', metric='cosine')

In [19]:
dir(model_knn)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_check_feature_names',
 '_check_n_features',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_pairwise',
 '_radius_neighbors_reduce_func',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_validate_data',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_features_in_',
 'n_jobs',
 'n_neighbors',
 'n_samples_fit_',
 

In [20]:
model_knn.n_neighbors

5

## one-vs-rest

In [21]:
def transform2(x):
    sc = StandardScaler()
    x_ = sc.fit_transform(x)
    
    return x_

## radius_neighbors Approach

In [22]:
r = np.arange(0.1, 1.1, 0.1)

for i in r:
    test_product = transform2(retrieve_stored_features(HOG_DIR, "Black Sugar Boba MIlk Tea").reshape(1, -1))

    distances, indices = model_knn.radius_neighbors(test_product, radius=i)

    print("R: {}".format(i))
    print(distances, indices, end="\n\n")

    
    
    
    
    
# for i in range(len(indices)):
#     idx = np.where(distances[i,]<1.9)[i]
# #     print(files[idx])
#     print(idx)

R: 0.1
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.2
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.30000000000000004
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.4
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.5
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.6
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.7000000000000001
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.8
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.9
[array([], dtype=float64)] [array([], dtype=int64)]

R: 1.0
[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])          

### Multiple products

In [23]:
distances, indices = model_knn.radius_neighbors(x[3:9], radius=0.1)

print("Distance: ", end="\n")
print(distances, end="\n\n")
print("Indices: ")
print(indices)

# for i in range(len(distances)):
#     idx = np.where(distances[i,]<0.8)[0] 
#     print(files[idx])
#     print(indices)
#     print(idx)

Distance: 
[array([1.11022302e-16]) array([4.4408921e-16])
 array([1.77635684e-15, 9.55086570e-02]) array([0.]) array([0.])
 array([0.])]

Indices: 
[array([3]) array([4]) array([ 5, 69]) array([6]) array([7]) array([8])]


### All products

In [24]:
distances, indices = model_knn.radius_neighbors(x, radius=0.1)

print(distances, indices)

# for i in range(len(distances)-1):
#     print(distances[i], indices[i])
#     if distances.flatten()[i] == 0:
#         print("Recommend: {} with distance {}".format(files[indices[0][i]].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

[array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.        , 0.09550866]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.        , 0.08906525]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.08906525, 0.        ]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.])
 array([0.09550866, 0.        ]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array

## kneighbors approach

### Single product

In [25]:
test_product = transform2(retrieve_stored_features(HOG_DIR, "Black Sugar Boba MIlk Tea").reshape(1, -1))

distances, indices = model_knn.kneighbors(test_product, n_neighbors=9)

for i in range(len(distances)):
#     print(distances[i], indices[i])
#     if i == 0:
#         print("Recommend: {} with distance {}".format(files[i].split(".")[0], distances.flatten()[i]))
#     else:
    print("{}: {} with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

0: Red Bean Almond Pudding with distance 1.0


### Multiple products

In [26]:
distances, indices = model_knn.kneighbors(x[3:9])

for i in range(len(distances)-1):
#     print(distances[i], indices[i])
#     if i == 0:
#         print("Recommend: {} with distance {}".format(files[i].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[i].split(".")[0], distances.flatten()[i]))
    print("{}: {} with distance \t\t {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

0: Hot Almond Soup Signature with distance 		 1.1102230246251565e-16
1: Double Taro Signature with distance 		 0.5117070816136311
2: Hot Almond Soup with distance 		 0.5614357267065564
3: Hot Grass Jelly Soup Signature with distance 		 0.6022747410926366
4: Icy Grass Jelly Signature with distance 		 0.6878594249537295


### All products

In [27]:
distances, indices = model_knn.kneighbors(x)

for i in range(len(distances)-1):
    print(distances[i], indices[i])
#     if distances.flatten()[i] == 0:
#         print("Recommend: {} with distance {}".format(files[indices[0][i]].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

[0.         0.73963037 0.75705925 0.76273086 0.79784045] [ 0 80 30  3 62]
[0.         0.56486904 0.61825171 0.63958029 0.65118737] [ 1  8 61 41 50]
[0.         0.50832069 0.60617253 0.66885645 0.72790503] [ 2 25 83 17 59]
[0.         0.51170708 0.56143573 0.60227474 0.68785942] [ 3 30 76 29 57]
[0.         0.27490113 0.42590834 0.45937768 0.51148383] [ 4 47 24 90 46]
[0.         0.09550866 0.49103456 0.5029351  0.66147136] [ 5 69 90 24 11]
[0.         0.51775915 0.55519568 0.61420665 0.62785491] [ 6 20 81  9 73]
[0.         0.11387515 0.14671342 0.16566173 0.16921671] [ 7 49 35 78 67]
[0.         0.56486904 0.60317377 0.64319064 0.64404489] [ 8  1 61 47 44]
[0.         0.57636056 0.584849   0.5897943  0.60083612] [ 9 20 81 16 12]
[0.         0.48178213 0.55941956 0.63800554 0.71485096] [10 17 40 48 73]
[0.         0.64043378 0.64204668 0.66147136 0.66460987] [11 15 66  5 90]
[0.         0.60083612 0.6033868  0.61448784 0.61600652] [12  9 44 20 37]
[0.         0.42698214 0.48549053 0.49